<a href="https://colab.research.google.com/github/BlackeWhite/SSD_Modanet/blob/master/SSD300/Traning_ssd_300.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Google Drive File

In [ ]:
#per salvare file di gorsse dimensioni sul drive 
!pip install httplib2==0.15.0
!pip install google-api-python-client==1.6
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
#pesi VGG_ILSVRC_16_layers_fc_reduced
Weight = drive.CreateFile({'id': '1sBmajn6vOE7qJ8GnxUJt4fGPuffVUZox'}) 
Weight.GetContentFile(Weight['title'])  # Save Drive file as a local file
print('Uploaded {}, id {}'.format(Weight['title'], Weight['id']))

#dataset
Modadat = drive.CreateFile({'id': '1il_eW1COHs1_QijMh3zklvhmE_Dfk0qh'}) 
Modadat.GetContentFile(Modadat['title'])  # Save Drive file as a local file
print('Uploaded {}, id {}'.format(Modadat['title'], Modadat['id']))
!unzip dataset.zip 
#annotazioni 
annotaz = drive.CreateFile({'id': '1aD4BgDD7grEFYlDKr0k_SHPw1qbt9N1q'}) 
annotaz.GetContentFile(annotaz['title'])  # Save Drive file as a local file
print('Uploaded {}, id {}'.format(annotaz['title'], annotaz['id']))
!unzip annotazioni.zip 

In [ ]:
!nvidia-smi

# Import library

Import library from github of Pireluigi Ferrari that had created the code for implement the SSD

In [ ]:
!pip install keras==2.2.3
#♦%tensorflow_version 1.x
!pip install tensorflow-gpu==1.15

import h5py
import numpy as np
import shutil
import os.path
if not(os.path.isdir('keras_layers')):
  !git clone https://github.com/pierluigiferrari/ssd_keras.git
  !mv ./ssd_keras/* . #copy all file in the root directory, you must this because after invocate the lib in the repository 
  !rm -r ssd_keras

from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TerminateOnNaN, CSVLogger
from keras import backend as K
from keras.models import load_model
import tensorflow as tf
from math import ceil
import numpy as np
from matplotlib import pyplot as plt

from sklearn.model_selection import train_test_split

from models.keras_ssd300 import ssd_300
from keras_loss_function.keras_ssd_loss import SSDLoss
from keras_layers.keras_layer_AnchorBoxes import AnchorBoxes
from keras_layers.keras_layer_DecodeDetections import DecodeDetections
from keras_layers.keras_layer_DecodeDetectionsFast import DecodeDetectionsFast
from keras_layers.keras_layer_L2Normalization import L2Normalization

from ssd_encoder_decoder.ssd_input_encoder import SSDInputEncoder
from ssd_encoder_decoder.ssd_output_decoder import decode_detections, decode_detections_fast

from data_generator.object_detection_2d_data_generator import DataGenerator
from data_generator.object_detection_2d_geometric_ops import Resize
from data_generator.object_detection_2d_photometric_ops import ConvertTo3Channels
from data_generator.data_augmentation_chain_original_ssd import SSDDataAugmentation, SSDRandomCrop
from data_generator.object_detection_2d_misc_utils import apply_inverse_transforms

import logging

%matplotlib inline

In [6]:
import logging
LOG_FILENAME = 'colab.log'

# Set the model configuration parameters

In [7]:
logging.debug('Create model configuration parameter')

# If you're resuming a previous training, set `initial_epoch` and `final_epoch` accordingly.
initial_epoch   = 0
final_epoch     = 120
steps_per_epoch = 256

batch_size=64
img_height = 300 # Height of the model input images
img_width = 300 # Width of the model input images
img_channels = 3 # Number of color channels of the model input images
mean_color = [123, 117, 104] # The per-channel mean of the images in the dataset. Do not change this value if you're using any of the pre-trained weights.
swap_channels = [2, 1, 0] # The color channel order in the original SSD is BGR, so we'll have the model reverse the color channel order of the input images.
n_classes = 13 # Number of positive classes
scales = [0.07, 0.15, 0.33, 0.51, 0.69, 0.87, 1.05] # The anchor box scaling factors used in the original SSD300 for the MS COCO datasets

aspect_ratios = [[1.0, 2.0, 0.5],
                 [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                 [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                 [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                 [1.0, 2.0, 0.5],
                 [1.0, 2.0, 0.5]] # The anchor box aspect ratios used in the original SSD300; the order matters
two_boxes_for_ar1 = True
steps = [8, 16, 32, 64, 100, 300] # The space between two adjacent anchor box center points for each predictor layer.
offsets = [0.5, 0.5, 0.5, 0.5, 0.5, 0.5] # The offsets of the first anchor box center points from the top and left borders of the image as a fraction of the step size for each predictor layer.
clip_boxes = False # Whether or not to clip the anchor boxes to lie entirely within the image boundaries
variances = [0.1, 0.1, 0.2, 0.2] # The variances by which the encoded target coordinates are divided as in the original implementation
normalize_coords = True
logging.debug('Parameters created')

# Build the model or load the model

Create a new model and load trained VGG-16 weights into it

In [ ]:
def Mean_IOU_tensorflow_2(y_true, y_pred):
    nb_classes = K.int_shape(y_pred)[-1]
    iou = []
    true_pixels = K.argmax(y_true, axis=-1)
    pred_pixels = K.argmax(y_pred, axis=-1)
    void_labels = K.equal(K.sum(y_true, axis=-1), 0)
    for i in range(0, nb_classes): # exclude first label (background) and last label (void)
        true_labels = K.equal(true_pixels, i) & ~void_labels
        pred_labels = K.equal(pred_pixels, i) & ~void_labels
        inter = tf.to_float(true_labels & pred_labels)
        union = tf.to_float(true_labels | pred_labels)
        legal_batches = K.sum(tf.to_int32(true_labels), axis=1)>0
        ious = K.sum(inter, axis=1)/K.sum(union, axis=1)
        iou.append(K.mean(ious[legal_batches]))
    iou = tf.stack(iou)
    legal_labels = ~tf.math.is_nan(iou)
    iou = iou[legal_labels]
    return -K.mean(iou)

def iou_loss(y_true,y_pred):
  iou=Mean_IOU_tensorflow_2(y_true,y_pred)
  return -iou

In [18]:

K.clear_session() # Clear previous models from memory.

model, predictor_sizes = ssd_300(image_size=(img_height, img_width, img_channels),
                n_classes=13,
                mode='training',
                l2_regularization=0.0005,
                scales=scales,
                aspect_ratios_per_layer=aspect_ratios,
                two_boxes_for_ar1=two_boxes_for_ar1,
                steps=steps,
                offsets=offsets,
                clip_boxes=clip_boxes,
                variances=variances,
                normalize_coords=normalize_coords,
                subtract_mean=mean_color,
                swap_channels=swap_channels,
                return_predictor_sizes=True)

# 2: Load some weights into the model.

# TODO: Set the path to the weights you want to load.
#weights_path = './ssd300_Modanet_data-aug_epoch-150_loss-5.3164_val_loss-5.1061.h5' #pesi all'epoca 150 
weights_path = './VGG_ILSVRC_16_layers_fc_reduced.h5' #pesi VGG


model.load_weights(weights_path, by_name=True)

# 3: Instantiate an optimizer and the SSD loss function and compile the model.
#    If you want to follow the original Caffe implementation, use the preset SGD
#    optimizer, otherwise I'd recommend the commented-out Adam optimizer.

#adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
sgd = SGD(lr=0.001, momentum=0.9, decay=0.0, nesterov=False)

ssd_loss = SSDLoss(neg_pos_ratio=3, alpha=1.0)


#model.compile(optimizer=adam, loss=ssd_loss.compute_loss, metrics=["acc"])
model.compile(optimizer=sgd, loss=ssd_loss.compute_loss,  metrics=["acc"])
logging.debug('Model preparation finisched')

# Load dataset

In [9]:
logging.debug('Load dataset')

In [10]:
def rename(path):   #funzione per apportare modifiche ai nomi del dataset 
	lis = os.listdir(path)
	for item in lis:
		temp=item.split(".")
		while len(temp[0])<7:
			new=str(0)+temp[0]
			temp[0]=new
			if len(temp[0])==7:
				os.rename(path+item,path+temp[0]+".jpg")

In [11]:
rename('./new_train/')
rename('./new_val/')
rename('./new_test/')

In [19]:
# TODO: Set the paths to your dataset here.
train_images_path = ['./new_train/']
train_labels_path = ['annotazioni_train.json']
val_images_path= ['./new_val/']
val_labels_path = ['annotazioni_val.json']

train_dataset = DataGenerator(load_images_into_memory=False)
val_dataset = DataGenerator(load_images_into_memory=False)

train_dataset.parse_json(images_dirs=train_images_path,
                  annotations_filenames=train_labels_path,
                  ground_truth_available= True,
                  include_classes = 'all',
                  ret=True)

val_dataset.parse_json(images_dirs=val_images_path,
                  annotations_filenames=val_labels_path,
                  ground_truth_available= True,
                  include_classes = 'all',
                  ret=True)

print("Number of images in the traning dataset:", train_dataset.get_dataset_size())
print("Number of images in the validation dataset:", val_dataset.get_dataset_size())

Processing 'annotazioni_val.json': 100%|██████████| 5225/5225 [00:00<00:00, 90557.71it/s]
Number of images in the traning dataset: 41810
Number of images in the validation dataset: 5225


In [20]:
# 3: Set the batch size.

batch_size = 64

# For the validation generator:
convert_to_3_channels = ConvertTo3Channels()
resize = Resize(height=img_height, width=img_width)

ssd_data_augmentation = SSDDataAugmentation(img_height=img_height,
                                            img_width=img_width,
                                            background=mean_color)

ssd_input_encoder = SSDInputEncoder(img_height=img_height,
                                    img_width=img_width,
                                    n_classes=n_classes,
                                    predictor_sizes=predictor_sizes,
                                    scales=scales,
                                    aspect_ratios_per_layer=aspect_ratios,
                                    two_boxes_for_ar1=two_boxes_for_ar1,
                                    steps=steps,
                                    offsets=offsets,
                                    clip_boxes=clip_boxes,
                                    variances=variances,
                                    matching_type='multi',
                                    pos_iou_threshold=0.5,
                                    neg_iou_limit=0.5,
                                    normalize_coords=normalize_coords)



# 6: Create the generator handles that will be passed to Keras' `fit_generator()` function.
train_generator = train_dataset.generate(batch_size=batch_size,
                                         shuffle=True,
                                         transformations=[convert_to_3_channels,resize],
                                         label_encoder=ssd_input_encoder,
                                         returns={'processed_images',
                                                  'encoded_labels'},
                                         keep_images_without_gt=False)
#rename('./new_train/')
val_generator = val_dataset.generate(batch_size=batch_size,
                                     shuffle=False,
                                     transformations=[convert_to_3_channels,resize],
                                     label_encoder=ssd_input_encoder,
                                     returns={'processed_images',
                                              'encoded_labels'},
                                     keep_images_without_gt=False)
# Get the number of samples in the training and validations datasets.
train_dataset_size = train_dataset.get_dataset_size()
val_dataset_size   = val_dataset.get_dataset_size()

print("Number of images in the training dataset:\t{:>6}".format(train_dataset_size))
print("Number of images in the validation dataset:\t{:>6}".format(val_dataset_size))

Number of images in the training dataset:	 41810
Number of images in the validation dataset:	  5225


In [13]:
logging.debug('Data loaded')

Set the remaining training parameters

In [21]:
logging.debug('Training preparation')
# Define a learning rate schedule.

def lr_schedule(epoch):
    if epoch < 80:
        return 0.001
    elif epoch < 100:
        return 0.0001
    else:
        return 0.00001
# Define model callbacks.

# TODO: Set the filepath under which you want to save the model.
model_checkpoint = ModelCheckpoint(filepath='/content/drive/My Drive/pesi_SSD/ssd300_Modanet_epoch-{epoch:02d}_loss-{loss:.4f}_val_loss-{val_loss:.4f}.h5',
                                   monitor='val_loss',
                                   verbose=1,
                                   save_best_only=True,
                                   save_weights_only=False,
                                   mode='auto',
                                   period=1)
#model_checkpoint.best = 

csv_logger = CSVLogger(filename='/content/drive/My Drive/log_SSD300.csv',
                       separator=',',
                       append=True)

learning_rate_scheduler = LearningRateScheduler(schedule=lr_schedule,
                                                verbose=1)

terminate_on_nan = TerminateOnNaN()

callbacks = [model_checkpoint,
             csv_logger,
             learning_rate_scheduler,
             terminate_on_nan]

logging.debug('Training prepared')

# Train the model

In [ ]:
logging.debug('Training start')



history = model.fit_generator(generator=train_generator,
                              steps_per_epoch=steps_per_epoch,
                              epochs=final_epoch,
                              callbacks=callbacks,
                              validation_data=val_generator,
                              validation_steps=ceil(val_dataset_size/batch_size),
                              initial_epoch=initial_epoch)

logging.debug('Training end')

Epoch 1/120

Epoch 00001: LearningRateScheduler setting learning rate to 0.001.
256/256 [==============================] - 448s 2s/step - loss: 12.0437 - acc: 0.3680 - val_loss: 8.7845 - val_acc: 0.3224

Epoch 00001: val_loss improved from inf to 8.78454, saving model to /content/drive/My Drive/pesi_SSD/ssd300_Modanet_epoch-01_loss-12.0436_val_loss-8.7845.h5


/usr/local/lib/python3.6/dist-packages/keras/utils/io_utils.py:186: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  self.data = h5py.File(path,)


Epoch 2/120

Epoch 00002: LearningRateScheduler setting learning rate to 0.001.
256/256 [==============================] - 408s 2s/step - loss: 8.3288 - acc: 0.3532 - val_loss: 7.8785 - val_acc: 0.3974

Epoch 00002: val_loss improved from 8.78454 to 7.87850, saving model to /content/drive/My Drive/pesi_SSD/ssd300_Modanet_epoch-02_loss-8.3288_val_loss-7.8785.h5
Epoch 3/120

Epoch 00003: LearningRateScheduler setting learning rate to 0.001.
256/256 [==============================] - 411s 2s/step - loss: 7.6731 - acc: 0.3869 - val_loss: 7.5045 - val_acc: 0.4237

Epoch 00003: val_loss improved from 7.87850 to 7.50454, saving model to /content/drive/My Drive/pesi_SSD/ssd300_Modanet_epoch-03_loss-7.6732_val_loss-7.5045.h5
Epoch 4/120

Epoch 00004: LearningRateScheduler setting learning rate to 0.001.
256/256 [==============================] - 411s 2s/step - loss: 7.3690 - acc: 0.4105 - val_loss: 7.2916 - val_acc: 0.4060

Epoch 00004: val_loss improved from 7.50454 to 7.29158, saving model to

# Save weight on drive  

In [ ]:
# Import PyDrive and associated libraries.
# This only needs to be done once in a notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.
auth.authenticate_user()
gauth = GoogleAuth()

gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Create & upload file.
uploaded = drive.CreateFile({'title': 'ssd300_Modanet_data-aug_epoch-30_loss-6.4659_val_loss-6.1568.h5'})
uploaded.SetContentFile('/content/ssd300_Modanet_data-aug_epoch-30_loss-6.4659_val_loss-6.1568.h5') #aggiungere path file
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

# Calcolo dell' IoU 


In [ ]:
predict_generator = val_dataset.generate(batch_size=1,
                                         shuffle=True,
                                         transformations=[convert_to_3_channels,
                                                          resize],
                                         label_encoder=None,
                                         returns={'processed_images',
                                                  'filenames',
                                                  'inverse_transform',
                                                  'original_images',
                                                  'original_labels'},
                                         keep_images_without_gt=False)

In [ ]:
from ssd_encoder_decoder.ssd_input_encoder import SSDInputEncoder
from ssd_encoder_decoder.ssd_output_decoder import decode_detections, decode_detections_fast
from data_generator.object_detection_2d_misc_utils import apply_inverse_transforms
from statistics import mean
classes = ['background',
           'bag', 'belt', 'boots', 'footwear',
           'outer', 'dress', 'sunglasses', 'pants',
           'top', 'shorts', 'skirt', 'headwear',
           'scarf/tie']
def iou_metric(y_true, y_pred):
    y_pred_decoded = decode_detections(y_pred,
                                   confidence_thresh=0.5,
                                   iou_threshold=0.4,
                                   top_k=200,
                                   normalize_coords=normalize_coords,
                                   img_height=img_height,
                                   img_width=img_width)
    y_pred_decoded_inv = apply_inverse_transforms(y_pred_decoded, batch_inverse_transforms)

    iou_list= []
    AoG = 0
    AoP = 0
    xmin_gt=0 
    ymin_gt=0
    xmax_gt=0
    ymax_gt = 0
    xmin_pred=0
    ymin_pred=0
    xmax_pred=0
    ymax_pred = 0

    for i in range(0,len(batch_original_labels)):
      for box in batch_original_labels[i]:
        for box_pred in y_pred_decoded_inv[i]:
          if (classes[int(box_pred[0])]==classes[int(box[0])]):
            xmin_gt = box[1]
            ymin_gt = box[2]
            xmax_gt = box[3]
            ymax_gt = box[4]
            AoG = abs(xmax_gt-xmin_gt) * abs(ymax_gt-ymin_gt)
            xmin_pred = box_pred[2]
            ymin_pred = box_pred[3]
            xmax_pred = box_pred[4]
            ymax_pred = box_pred[5]
            AoP = abs(xmax_pred-xmin_pred) * abs(ymax_pred-ymin_pred)
            
            # overlaps are the co-ordinates of intersection box
            overlap_0 = max(xmin_gt, xmin_pred)
            overlap_1 = min(xmax_gt, xmax_pred)
            overlap_2 = max(ymin_gt, ymin_pred)
            overlap_3 = min(ymax_gt, ymax_pred)

            # intersection area
            intersection = abs((overlap_3 - overlap_2) * (overlap_1 - overlap_0))
            # area of union of both boxes
            union = AoG + AoP - intersection

            # iou calculation
            temp=intersection/union
            iou_list.append(temp)
    #print(iou_list)
    if not(iou_list ==[]):
      iou=mean(iou_list)
      return iou
     

In [ ]:
iou_list=[]
for i in range(0,val_dataset.dataset_size):
  batch_images, batch_filenames, batch_inverse_transforms, batch_original_images, batch_original_labels = next(predict_generator)
  y_pred = model.predict(batch_images)
  iou=iou_metric(batch_original_labels,y_pred)
  iou_list.append(iou)  

In [ ]:
print(iou_list)
lungh=len(iou_list)
print(lungh)
somma=0
for i in iou_list:
  if not i is None: 
    somma+=i
print("L'IoU media è " + str(somma/lungh))

# Generazione dei bounding box predetti e reali 

In [ ]:
y_pred = model.predict(batch_images)

y_pred_decoded = decode_detections(y_pred,
                                   confidence_thresh=0.5,
                                   iou_threshold=0.4,
                                   top_k=200,
                                   normalize_coords=normalize_coords,
                                   img_height=img_height,
                                   img_width=img_width)

In [ ]:
y_pred_decoded_inv = apply_inverse_transforms(y_pred_decoded, batch_inverse_transforms)

np.set_printoptions(precision=2, suppress=True, linewidth=90)
print("Predicted boxes:\n")
print('   class   conf xmin   ymin   xmax   ymax')
print(y_pred_decoded_inv[i])

In [ ]:
colors = plt.cm.hsv(np.linspace(0, 1, n_classes+1)).tolist()

classes = ['background',
           'bag', 'belt', 'boots', 'footwear',
           'outer', 'dress', 'sunglasses', 'pants',
           'top', 'shorts', 'skirt', 'headwear',
           'scarf/tie']

plt.figure(figsize=(20,12))
plt.imshow(batch_original_images[i])

current_axis = plt.gca()

for box in batch_original_labels[i]:
    xmin = box[1]
    ymin = box[2]
    xmax = box[3]
    ymax = box[4]
    label = '{}'.format(classes[int(box[0])])
    current_axis.add_patch(plt.Rectangle((xmin, ymin), xmax-xmin, ymax-ymin, color='green', fill=False, linewidth=2))  
    current_axis.text(xmin, ymin, label, size='x-large', color='white', bbox={'facecolor':'green', 'alpha':1.0})

for box in y_pred_decoded_inv[i]:
    xmin = box[2]
    ymin = box[3]
    xmax = box[4]
    ymax = box[5]
    color = colors[int(box[0])]
    label = '{}: {:.2f}'.format(classes[int(box[0])], box[1])
    current_axis.add_patch(plt.Rectangle((xmin, ymin), xmax-xmin, ymax-ymin, color=color, fill=False, linewidth=2))  
    current_axis.text(xmin, ymin, label, size='x-large', color='white', bbox={'facecolor':color, 'alpha':1.0})